<a href="https://colab.research.google.com/github/DongTianKolmostar/colab-data/blob/main/Test_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Catalogue


# Utils

In [ ]:
#@title Basic Func
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from numpy import pi
from typing import *
from google.colab import widgets as cwidgets

import os
from os import path as op

from plotly import io as pio
pio.renderers.default = 'colab'

DRAW_CONF = dict(
    line=dict(width=1),
    marker=dict(size=3),
    mode='lines+markers',
    hovertemplate='(%{x:.8s}, %{y:.3s})'
)

def rf_data_decode(data_hex: str, quant=2):
  '''
  each block has 514 words:
      512 words IQ data;
      1 word ADC counter;
      1 word TCXO counter.
  '''
  data_u32 = np.array([int(_hex, 16)
                       for _hex in data_hex.split(' ') if _hex])

  n_word = len(data_u32)
  if n_word % 514:
    raise Exception(f'n_word % 514 = {n_word} % 514 '
                    f'= {n_word % 514}')
  blk_num = n_word // 514
  iq_bits = 2 << quant
  iq_mask = (1 << iq_bits) - 1
  iq_in_word = 32 // iq_bits
  iq_u32 = np.zeros(512 * blk_num, dtype=int)
  counter = np.zeros((blk_num, 2), dtype=int)
  for i in range(blk_num):
    iq_u32[512*i:512*i+512] = data_u32[514*i:514*i+512]
    counter[i, 0] = data_u32[514*i+512]
    counter[i, 1] = data_u32[514*i+513]
  iq = np.zeros(512 * blk_num * iq_in_word, dtype=float)
  for i in range(iq_in_word):
    iq[i::iq_in_word] = ((iq_u32 >> (iq_bits * i))
                         & iq_mask) - iq_mask/2

  xt = iq[::2] + 1j * iq[1::2]
  return xt, counter

def download_file(fpath: str):
  dirname = op.dirname(fpath)
  if dirname:
    os.makedirs(dirname, exist_ok=True)
  url = ('https://media.githubusercontent.com/media/'
         'DongTianKolmostar/colab-data/main')
  cmd = 'curl {0}/{1} -o ./{1}'.format(url, fpath)
  print(cmd)
  os.system(cmd)

def rf_from_file(fpath: str, quant=2, count=-1):
  iq_bits = 2 << quant
  iq_mask = (1 << iq_bits) - 1
  iq_in_word = 32 // iq_bits

  iq_u32 = np.fromfile(fpath, dtype='<u4',
                       count=count * 2 // iq_in_word)
  iq = np.zeros(len(iq_u32) * iq_in_word, dtype=float)
  for i in range(iq_in_word):
    iq[i::iq_in_word] = ((iq_u32 >> (iq_bits * i))
                         & iq_mask) - iq_mask/2

  xt = iq[::2] - 1j * iq[1::2]
  return xt

def plot(x=None, y=None, logy=False, name='', title='',
         xlabel=None, ylabel=None) -> go.Figure:
  xlen = len(y)
  if xlen > 1e4:
    from plotly.graph_objects import Scattergl as Scatter
  else:
    from plotly.graph_objects import Scatter
  fig = go.Figure(Scatter(x=x, y=y, name=name, **DRAW_CONF))
  fig.update_layout(title=title)
  if logy:
    fig.update_yaxes(type="log")
  if xlabel:
    fig.update_xaxes(title_text=xlabel)
  if ylabel:
    fig.update_yaxes(title_text=ylabel)
  return fig

def show(figs: Dict[str, go.Figure]):
  klst = list(figs.keys())
  vlst = list(figs.values())
  fig_num = len(klst)
  tab = cwidgets.TabBar(klst)
  for i in range(fig_num):
    with tab.output_to(i, select=False):
      fig = vlst[i]
      fig.show()
  return tab

def plot_xt(xt, sps, xtime=False, angle=False,
            title_prefix='') -> go.Figure:
  xlen = len(xt)
  if xlen > 1e4:
    from plotly.graph_objects import Scattergl as Scatter
  else:
    from plotly.graph_objects import Scatter

  if xtime:
    ts = np.linspace(0, xlen/sps, xlen, endpoint=False)
    xlabel = 'Time / s'
  else:
    ts = np.arange(xlen)
    xlabel = 'Sample Index'

  fig = go.Figure([Scatter(x=ts, y=xt.real, name='I', **DRAW_CONF),
                   Scatter(x=ts, y=xt.imag, name='Q', **DRAW_CONF)])
  fig.update_layout(title=title_prefix + 'Signal')
  fig.update_xaxes(title_text=xlabel)
  fig.update_yaxes(title_text='Amplitude')

  if angle:
    ang = np.unwrap(np.angle(xt))
    fig1 = go.Figure([Scatter(x=ts, y=ang, name='xt', **DRAW_CONF)])
    fig1.update_layout(title=title_prefix + 'Angle')
    fig1.update_xaxes(title_text=xlabel)
    fig1.update_yaxes(title_text='Angle / rad')
    return fig, fig1
  return fig

def calc_xf(xt, sps):
  xlen = len(xt)
  fs = np.fft.fftfreq(xlen, 1/sps)
  xf = np.fft.fft(xt)
  fs = np.fft.fftshift(fs)
  xf = np.fft.fftshift(xf)
  return fs, xf

def plot_xf(fs, xf, title_prefix='') -> go.Figure:
  xlen = len(xf)
  if xlen > 1e4:
    from plotly.graph_objects import Scattergl as Scatter
  else:
    from plotly.graph_objects import Scatter

  abs_xf = abs(xf)
  fig = go.Figure(Scatter(x=fs, y=abs_xf, name='xf', **DRAW_CONF))
  fig.update_layout(title=title_prefix + 'FFT')
  fig.update_xaxes(title_text='Freq / Hz')
  fig.update_yaxes(title_text='|X(f)|', type="log")
  return fig

def calc_angle_diff(xt, freq, sps):
  ns = np.arange(len(xt))
  ang = np.unwrap(np.angle(xt))
  diff = ang - ang[0] - ns * freq / sps * 2 * np.pi
  return diff

def plot_angle_diff(xt, freq, sps, title_prefix=''):
  diff = calc_angle_diff(xt, freq, sps)
  return plot(y=diff, name='xt', title=title_prefix + 'Angle Diff',
              xlabel='Sample Index', ylabel='Angle / rad')

def rng_ifft(fs, xf, center, width):
  rng = (fs > center - width) & (fs < center + width)
  xf = xf.copy()
  xf[~rng] = 0
  _xf = np.fft.fftshift(xf)
  xt = np.fft.ifft(_xf)
  return xt, xf

def find_peak_idx(fs, xf, freq, width):
  abs_xf = abs(xf)
  rng = (fs > freq - width) & (fs < freq + width)
  print(np.count_nonzero(rng))
  op_idx = np.argwhere(rng)[0]
  imax = op_idx + np.argmax(abs_xf[rng])
  return imax

def calc_freq_offset(xt, freq, sps):
  fs, xf = calc_xf(xt, sps)
  imax = find_peak_idx(fs, xf, freq, 0.01 * 1.023e6)
  abs_xf = abs(xf)
  if abs_xf[imax-1] > abs_xf[imax+1]:
    imax -= 1
  k = range(imax, imax+2)
  pk_freq = sum(fs[k] * abs_xf[k]) / sum(abs_xf[k])
  offset = pk_freq - freq
  
  xlen = len(xt)
  ts = np.linspace(0, xlen/sps, xlen, endpoint=False)
  xt *= np.exp(-2j * pi * offset * ts)

  print('Freq Offset:', offset)
  return offset, xt

def calc_snr(fs, xf, freq):
  imax = find_peak_idx(fs, xf, freq, 0.01 * 1.023e6)
  abs_xf = abs(xf)
  print('Peak:', fs[imax], abs_xf[imax])
  if abs_xf[imax-1] > abs_xf[imax+1]:
    imax -= 1
  power_all = sum(abs_xf ** 2)
  power_pk = sum(abs_xf[imax:imax+2] ** 2)
  power_noise = power_all - power_pk
  snr = 10 * np.log10(power_pk / power_noise)
  print('SNR :', snr, 'dB')
  return snr

def notch_filter(osig: np.ndarray, sps: float,
                 freq_offset: float, period: int):
    xlen = len(osig)
    ts = np.linspace(0, xlen / sps, xlen, endpoint=False)
    sig_offset = osig * np.exp(-2j * np.pi * freq_offset * ts)

    n_period = xlen // period
    avg = np.mean(np.reshape(sig_offset[:period * n_period], (-1, period)), 0)

    sig_filter = sig_offset - np.tile(avg, n_period + 1)[:xlen]

    return sig_filter * np.exp(2j * np.pi * freq_offset * ts)


In [ ]:
#@title RF Data Decode Test
data_hex = "AD749964 A3A1AC88 74AB8CB0 5E85609F 7759636C A16B8D5B A29BA681 79A892AC 5B8762A0 715E676E 9A5F815E A594A87B 7EAA9BA2 5D8863A5 665C5E73 945B7C53 A68DA173 83AD9FA7 5D8B5FA1 655E5B77 99607956 AB8FAA78 8BAAA09F 5B9372A5 665B5473 98638759 A3819F6A 7FAE9BA1 598C65A2 6E6A5B78 955A7C5E A98BA76F 7DA79C9F 5B83629B 695C5B74 975A7D58 A592AB71 83A796A3 5685639F 6C595D6E 99618354 A09FA47A 79AE92AD 528B61A1 69605868 A26C8358 9F9EA685 6DA98BAE 597C5B98 6E5B5B64 9F6D8D5E 9AA5A083 69A280B0 56745A8E 7D5C645D A9789C5F 8FA6A297 5E9C74AA 606E5D86 895B7259 A1819567 82B19BA7 5B966AAA 66605975 93677766 A29AA979 77AE8EA6 558D5EA5 6C5B576D 9F66845D 9FA0AA81 6BAD89AC 597F5B9A 75566165 A47A9162 94A3A795 5F9E79AB 606B5586 895C7559 A9859C64 8BB0A0A5 56966AAD 6D5F5B7A 965E8158 A499A87A 7CAE90A8 598169A0 735B6066 A1658E5B 9AA3A685 65A885AC 5773538D 825C6460 A57B9964 94A4A28C 629E78AA 5F685D81 8C5A745C A98CA86B 84AA9FA2 569568A6 675C587B 955F8055 AA96A476 7CAE95AD 53865F9F 7B5B6069 9E699057 9BA3A487 6BA685AF 5D745A8C 805A6964 AA7D9B60 93A8A596 5F9A7BAC 60695880 8C5D735B A885A169 83B49AA3 5D9668AE 69635B7E 9867895B AD9FAA7B 7FA897AA 5A8A67A3 755F616F 9D6E8D5E 9AA4A589 70AB8BB0 5982609A 76676067 AA749360 9BA0A78C 6BAB84AD 577B5D9C 7E5D6764 AB7C9C69 96A3A792 65A97EB0 617B5C8F 83596B63 A57A9A66 92AF9F98 67AA79B7 59775E8F 885E6960 A07B9B69 90AE9E9A 5FA67AB1 6076568C 875A6D5C A77C9E69 8AA69C95 64A175B2 5B715683 8C5F7262 A6809D6B 87AFA19E 5FA273B2 5C6B5E88 86596A5E A780A365 90B09F98 5FA172AF 5B715389 87586E5A A1789762 90ADA69E 649F71A5 57715A91 7C55655A A877975F 9AA1A48D 65A27EAD 5376568A 71585E5F 99718860 96A6A18D 68A97EAE 56825B95 77556363 A16E8D5E 999EA78F 70AB86AC 5A815E9D 6F565D6C 9E638755 9799A47F 7CAC8DA3 618D629E 69605D74 9266815C A997A679 84B09AA8 5E9B6CA9 62615B7A 8F64785C A381A06D 83B1A19A 61986BA8 57705288 815F6C58 A076976E 8FAAA38C 66A37BAD 54755A8F 7D56665F 9B789663 9BA19F8B 6AA07DAA 55805C9C 71596068 A1689158 A393A57C 7AA993A2 5D8E6D9C 60605870 905C7558 A489A371 86ADA09F 5DA26BAD 5D5E5581 8B606F5A A57F9868 96A8A09C 63A67CAE 5B775991 7C646767 A6719565 98A5A68A 69A885A9 58795994 6E535F64 9F6C8A58 A89EA589 76AF8CAC 558C62A5 6E655B72 9E66895F A596A37C 81AE94AA 5A926CA4 60665380 985D7F59 A48BA269 8AAF9CA1 5EA270AD 61635B86 885B7459 AB7DA067 94B2A49B 5FA479B2 646F5A8C 7F557061 AB729D5F 9AA4AC8E 6EB085AA 65795F9A 7D5E6B64 A3729866 9BA5A488 73AF88AF 587C5C9E 7C5B6C66 A8689458 9D9AA888 6EA987A6 598260A1 6D5B6268 A46B8857 9E98A483 7BB095A8 558263A4 70585A63 9B66865C A298A67E 73AA8DAC 5880629C 6E525E60 9C678659 9F9EA082 78A68CA9 557A5C9B 76565D66 9E678C57 A69FAA84 71A390AA 57836198 74586168 A3678D5A 9B92A585 66AA85A7 5D7C5B8F 77575E65 A26A9062 99A2A781 67A77FA8 53765595 73565E5F A1789161 92AEA591 65A07CAE 5768518D 834F6858 A4799D61 89A9A49E 5B9D75AB 5F6E5784 8C5C7263 A58AA172 84AD9C99 569C6BAB 6063577E 95577D5B A28FA571 79AB94AA 538F5E9F 6A5D5970 A1678356 A299A57C 72AE8FAA 5B835FA0 6C605F6D A46C8D5D 9F9FA681 6AA283AC 5C715D8B 7F5B6763 A4779361 94A9A499 5CA07BA9 576B5186 85597259 9C809965 83A6919F 57996BA8 6160527E 945E785A A292A677 76AE93A9 548C5CA3 6B5F5B6D 9A69895D 9F9BA37C 70A685AC 5779599B 7E5D6865 A6799764 95A2A795 63A27CA8 586E568B 855C6E62 A37F9E60 85AE9B99 58A070B1 62685983 96577C57 A888A967 85AE9A9B 598F6AA9 675B5D6E 9D658758 A798AC79 76A891AA 54855D99 79545F67 A673975A 9F9EA78A 66A781A8 59745894 825A695C A677995F 96A9A496 6CA27BAB 64675D8E 805D6F60 AD839E6B 8FAAA59B 5D9770A6 5F655385 925E7B5E AE92A973 85AC9EA3 5C9167A7 635D577A 98647A5A AC94AB78 7EAA96A8 599869A8 6C5B5877 9D6D855E A19AA882 81AE9AA1 5E896BA5 6E636171 9967835D A49CA77D 73AC92A8 5B8D5DA1 76646277 9C708765 AA9DA97D 72AE92B4 588F659F 71615D6E A166825B A898A87F 7AA792A8 528C61A6 705D5D73 99688B55 9B9BA286 78AE90A9 538C60A4 67625A7C 9C628A54 A29BA274 7FA390A6 5C8A679F 67615B74 8D66795F A690A075 7EAB94A5 579867A8 65655773 96627759 A28DA372 81AD91A6 5C9371A8 5B61557A 8F607654 9F83936C 89AE9CA2 5C9D71AA 5971588B 82506B58 A1789E60 93A49E95 64A27CAA 56725791 7652605F A16E935F 9B9DA488 70A888A7 57876097 6957526C A1688E5D 9C95A27D 80A996A7 538E62A3 5D5D5771 965E7657 A887A272 83AE9EA1 599C69B0 5C6B5383 8759725B A97C9E64 95A7A494 6BAB81B0 58785896 7D586563 9F6B965C 9E9FA87F 71A888A4 548C5AA3 685E556D 975E8158 A792A66F 87AA989F 5F9D6BAC 5E655782 8A5B7457 9C769463 98AEA397 6DA885AB 597D5798 755A5F67 A4739468 9F9EA882 77AB8FAB 548560A3 745F5B6D 9B5D8859 A987A774 81AC9A9E 5A996CAA 5B6C5580 8B616F5F A783A066 90A59F96 60A477AF 5472568F 7559625E A068905B 9FA3A48A 69AD88AC 5C895B9C 6E55606B 9C6A825E A097A481 76AD8EA9 579C6AA8 6963537D 945D855F A689A26E 8AB09EA0 5D9D73AE 5E725684 8A59725D A8819F64 97A6A094 00003B8B 00001804 " #@param {type:"string"}
sps = 40 * 1.023e6
quant = 2
xt_freq = -4 * 1.023e6
xt_power = -80

xt, cnt = rf_data_decode(data_hex, quant)
g0 = plot_xt(xt[:1024], sps)
g1 = plot_angle_diff(xt[:1024], xt_freq, sps)

# offset, xt = calc_freq_offset(xt, xt_freq, sps)
fs, xf = calc_xf(xt, sps)
g2 = plot_xf(fs, xf)

tab = show({
    'g0': g0,
    'g1': g1,
    'g2': g2,
})

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>